# Mini-Lab: Logistic Regression and SVMs

Names:
Dylan Scott
Jobin Joseph
Nnenna Okpara
Satvik Ajmera

Instructions:
You are to perform predictive analysis (classification) upon a data set: model the dataset using
methods we have discussed in class: logistic regression and support vector machines, and making
conclusions from the analysis. Follow the CRISP-DM framework in your analysis (you are not
performing all of the CRISP-DM outline, only the portions relevant to the grading rubric outlined
below). This report is worth 10% of the final grade. You may complete this assignment in teams of
as many as three people.

Write a report covering all the steps of the project. The format of the document can be PDF,
*.ipynb, or HTML. You can write the report in whatever format you like, but it is easiest to turn in the
rendered iPython notebook. The results should be reproducible using your report. Please carefully
describe every assumption and every step in your report.

SVM and Logistic Regression Modeling
• [50 points] Create a logistic regression model and a support vector machine model for the
classification task involved with your dataset. Assess how well each model performs (use
80/20 training/testing split for your data). Adjust parameters of the models to make them more
accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel
only is fine to use.

[pick performance stats]

• [10 points] Discuss the advantages of each model for each classification task. Does one type
of model offer superior performance over another in terms of prediction accuracy? In terms of
training time or efficiency? Explain in detail.

• [30 points] Use the weights from logistic regression to interpret the importance of different
features for each classification task. Explain your interpretation in detail. Why do you think
some variables are more important?

• [10 points] Look at the chosen support vectors for the classification task. Do these provide
any insight into the data? Explain.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

import plotly.express as px
import plotly.graph_objects as go

## Dataset update:
For this project, we decided on using data directly for the NTSB database. The new data required us to convert the `mdb` files to multiple csvs. From there we were tasked with concatenating, and joining several CSVs. This allowed us to include new variables for prediction such as aircraft weight, ground injuries and weather to predict the response variable. By using the data directly from the NTSB database, it allowed for to include the most recent aviation incidents recorded to date. So we are left with 115,706 observations compared to the 90,000 observations from Lab 1. This next section will be further cleaning up our data before running any of our models.

## Data Cleaning

Upon importing our data, extraneous columns were to be removed because some were repeated or not useful to our model.

In [2]:
#Read in the Aviation Data
final_data = pd.read_csv("Data/final_data.csv",low_memory=False,dtype={'damage': str})
#Delete columns that were imported incorrectly
del final_data["Unnamed: 0"]
del final_data["dprt_state.1"]
del final_data["index"]
del final_data["ntsb_no_x"]
del final_data["ev_id"]
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115706 entries, 0 to 115705
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   acft_make          115643 non-null  object 
 1   acft_model         115630 non-null  object 
 2   cert_max_gr_wt     98673 non-null   float64
 3   acft_category      115287 non-null  object 
 4   damage             113877 non-null  object 
 5   far_part           114925 non-null  object 
 6   afm_hrs_last_insp  60298 non-null   float64
 7   type_fly           108599 non-null  object 
 8   dprt_city          111864 non-null  object 
 9   dprt_state         108791 non-null  object 
 10  rwy_len            64222 non-null   float64
 11  rwy_width          63110 non-null   float64
 12  ev_type            115706 non-null  object 
 13  ev_city            115646 non-null  object 
 14  ev_state           109635 non-null  object 
 15  ev_country         115199 non-null  object 
 16  ev

## Data Cleaning Explained

We fixed the levels of columns that should have been grouped together for the `damage` predictor. We fixed the levels of the `ev_city`, by making all the city names uppercase so they would be grouped together. For `wind_vel_ind` the unknown values were labeled as `U` and `UNK` to `UNK`. Then, we took the subset of the continuous predictors: `cert_max_gr_wt`,`afm_hrs_last_insp`,`rwy_len`, and `rwy_width` and removed all the NAs. We were left with approximately 35,000 rows that would be usable for prediction.

The NTSB database allowed use to include ground injuries in addition to flight injuries. So we renamed the columns, replaced the NAs in these columns with 0 and summed all them to get a total injuries column. We wanted to only include the total injuries and drop the other types of injury columns. From there, we converted the total injuries column to a binary variable for classification. If the total injuries were greater than 1, it is considered injured and less than 1 is uninjured. Our last step was to impute all the NA's for the categorical values with an unknown value called `UNK` In all, we are left with the dataframe `final_df` which consisted of 21 columns and 35,427 observations. 

Of the 35,427 observations, 18750 observations were reported as injured and 16677 were reported as uninjured. This column called `Injury` is the response variable, we would like to predict for the classification problem. 

In [3]:
#It looks like we have some missing values and have an inconsistant UNK vs UNK on flight damage
#combine all injuries includigng those on the ground
#sky_cond_ceil, sky_cond_nonceil
#chekc U vs Unk for wind_vel_ind
#flight crew 
finaldamagecount = final_data["damage"].value_counts().reset_index()
finaldamagecount.head(5)

,index,damage
0,SUBS,87994
1,DEST,20892
2,MINR,3302
3,NONE,1600
4,UNK,45


In [4]:
#looks like we have some inconsistant cities since some are upper and some are lower case
final_data['ev_city'] = final_data['ev_city'].str.upper()
ev_city_fix = final_data["ev_city"].value_counts().reset_index()
ev_city_fix.head(10)

,index,ev_city
0,ANCHORAGE,358
1,ANCHORAGE,286
2,FORT LAUDERDALE,206
3,FAIRBANKS,203
4,PHOENIX,202
5,HOUSTON,196
6,MIAMI,188
7,ATLANTA,186
8,RENO,185
9,TALKEETNA,182


In [5]:
#looks like we have some inconsistant cities since some are upper and some are lower case
final_data['ev_city'] = final_data['ev_city'].str.upper()
ev_city_fix = final_data["ev_city"].value_counts().reset_index()
ev_city_fix.head(10)

,index,ev_city
0,ANCHORAGE,358
1,ANCHORAGE,286
2,FORT LAUDERDALE,206
3,FAIRBANKS,203
4,PHOENIX,202
5,HOUSTON,196
6,MIAMI,188
7,ATLANTA,186
8,RENO,185
9,TALKEETNA,182


In [6]:
final_data.loc[final_data['damage'].str.contains('UNK', na=False), 'damage'] = 'UNK'
finaldamagecount = final_data["damage"].value_counts().reset_index()
finaldamagecount.head(5)

,index,damage
0,SUBS,87994
1,DEST,20892
2,MINR,3302
3,NONE,1600
4,UNK,89


In [7]:
#checking to see if wind_vel_ind had a miss-match with U and UNK
wind_count = final_data["wind_vel_ind"].value_counts().reset_index()
wind_count.head(6)

,index,wind_vel_ind
0,F,47663
1,UNK,42470
2,SPEC,10900
3,CALM,7074
4,T,4030
5,LVAR,1486


In [8]:
#dealing with unknnowns
#some columns we can't simply replace the blank value with "Unknown" or 0s since that will skew our data
#'cert_max_gr_wt','afm_hrs_last_insp','rwy_len','rwy_width'
# with the columns listed above we have elected to remove any rows where they are blank. This will help focus our data and it will still leave us with an ample amount of data
final_data.dropna(subset=['cert_max_gr_wt','afm_hrs_last_insp','rwy_len','rwy_width'],inplace=True)

In [9]:
#rename the injuries columns to make them easier to read
final_data = final_data.rename(columns={"inj_tot_f": "Total_Fatal_Injuries", "inj_tot_s": "Total_Serious_Injuries","inj_tot_m":"Total_Minor_Injuries","inj_tot_n":'Total_Uninjured',"inj_tot_t":"Total_Injuries_Flight"})

#fill in 0s when there wasn't an injury in that category
final_data.update(final_data[['Total_Fatal_Injuries','Total_Serious_Injuries','Total_Minor_Injuries','Total_Uninjured','Total_Injuries_Flight','inj_f_grnd','inj_m_grnd','inj_s_grnd']].fillna(0))
final_data.head()

,acft_make,acft_model,cert_max_gr_wt,acft_category,damage,far_part,afm_hrs_last_insp,type_fly,dprt_city,dprt_state,...,Total_Fatal_Injuries,Total_Minor_Injuries,Total_Uninjured,Total_Serious_Injuries,Total_Injuries_Flight,sky_cond_ceil,sky_cond_nonceil,wind_vel_ind,wx_int_precip,phase_flt_spec
1,Boeing,747-100,750000.0,AIR,MINR,121,113.0,UNK,CHITOSE,JA,...,0.0,0.0,4.0,0.0,0.0,NONE,SCAT,CALM,UNK,Landing
3,Cessna,172,2300.0,AIR,SUBS,091,40.0,PERS,,,...,0.0,0.0,1.0,0.0,0.0,BKN,UNK,UNK,LGT,Unknown
4,Cessna,207,3800.0,AIR,SUBS,135,49.0,UNK,,AK,...,0.0,0.0,1.0,0.0,0.0,BKN,UNK,UNK,UNK,Descent
6,Beech,300,14100.0,AIR,DEST,091,3.0,EXEC,GREENEVILLE,SC,...,2.0,0.0,0.0,0.0,2.0,BKN,UNK,UNK,MOD,Approach
8,Aero Commander,560A,6000.0,AIR,DEST,091,13.0,PERS,,,...,2.0,0.0,0.0,2.0,4.0,NONE,CLER,UNK,UNK,Approach


In [10]:
#set missing variables to Unknown in order to run our models
final_data.update(final_data.fillna("UNK"))
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35427 entries, 1 to 115696
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   acft_make               35427 non-null  object 
 1   acft_model              35427 non-null  object 
 2   cert_max_gr_wt          35427 non-null  float64
 3   acft_category           35427 non-null  object 
 4   damage                  35427 non-null  object 
 5   far_part                35427 non-null  object 
 6   afm_hrs_last_insp       35427 non-null  float64
 7   type_fly                35427 non-null  object 
 8   dprt_city               35427 non-null  object 
 9   dprt_state              35427 non-null  object 
 10  rwy_len                 35427 non-null  float64
 11  rwy_width               35427 non-null  float64
 12  ev_type                 35427 non-null  object 
 13  ev_city                 35427 non-null  object 
 14  ev_state                35427 non-nul

We will be using code from this classes Github: 
https://github.com/jakemdrew/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb

In [11]:
#we want to account for ALL injuries. This includes injuries on the ground as well as passangers
#Here we will make a new column that shows total injuries including ground ones
final_data['Total_Injuries_Ground'] = final_data['inj_f_grnd']+final_data['inj_m_grnd']+final_data['inj_s_grnd']
final_data['Total_Injuries'] = final_data['Total_Injuries_Ground']+final_data['Total_Injuries_Flight']
final_data.head()

,acft_make,acft_model,cert_max_gr_wt,acft_category,damage,far_part,afm_hrs_last_insp,type_fly,dprt_city,dprt_state,...,Total_Uninjured,Total_Serious_Injuries,Total_Injuries_Flight,sky_cond_ceil,sky_cond_nonceil,wind_vel_ind,wx_int_precip,phase_flt_spec,Total_Injuries_Ground,Total_Injuries
1,Boeing,747-100,750000.0,AIR,MINR,121,113.0,UNK,CHITOSE,JA,...,4.0,0.0,0.0,NONE,SCAT,CALM,UNK,Landing,0.0,0.0
3,Cessna,172,2300.0,AIR,SUBS,091,40.0,PERS,,,...,1.0,0.0,0.0,BKN,UNK,UNK,LGT,Unknown,0.0,0.0
4,Cessna,207,3800.0,AIR,SUBS,135,49.0,UNK,,AK,...,1.0,0.0,0.0,BKN,UNK,UNK,UNK,Descent,0.0,0.0
6,Beech,300,14100.0,AIR,DEST,091,3.0,EXEC,GREENEVILLE,SC,...,0.0,0.0,2.0,BKN,UNK,UNK,MOD,Approach,0.0,2.0
8,Aero Commander,560A,6000.0,AIR,DEST,091,13.0,PERS,,,...,0.0,2.0,4.0,NONE,CLER,UNK,UNK,Approach,0.0,4.0


In [12]:
#create a new column of injuried or not to get a binary response
#1 means someone was hurt 0 means someone was not
final_data['Injury'] = np.where(final_data['Total_Injuries'] >0,1,0)
injuries = final_data["Injury"].value_counts().reset_index()
injuries.head(2)

,index,Injury
0,1,18750
1,0,16677


In [13]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35427 entries, 1 to 115696
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   acft_make               35427 non-null  object 
 1   acft_model              35427 non-null  object 
 2   cert_max_gr_wt          35427 non-null  float64
 3   acft_category           35427 non-null  object 
 4   damage                  35427 non-null  object 
 5   far_part                35427 non-null  object 
 6   afm_hrs_last_insp       35427 non-null  float64
 7   type_fly                35427 non-null  object 
 8   dprt_city               35427 non-null  object 
 9   dprt_state              35427 non-null  object 
 10  rwy_len                 35427 non-null  float64
 11  rwy_width               35427 non-null  float64
 12  ev_type                 35427 non-null  object 
 13  ev_city                 35427 non-null  object 
 14  ev_state                35427 non-nul

In [14]:
final_df = final_data.copy()
#Since we added up all of our injuries we don't need the other columns that include injury count since it will be colinear to our prediction variable
final_df = final_df.drop(['Total_Fatal_Injuries','Total_Serious_Injuries','Total_Minor_Injuries','Total_Uninjured','Total_Injuries_Flight','inj_f_grnd','inj_m_grnd','inj_s_grnd','Total_Injuries_Ground',"Total_Injuries", "dprt_city"],axis = 1)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35427 entries, 1 to 115696
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   acft_make          35427 non-null  object 
 1   acft_model         35427 non-null  object 
 2   cert_max_gr_wt     35427 non-null  float64
 3   acft_category      35427 non-null  object 
 4   damage             35427 non-null  object 
 5   far_part           35427 non-null  object 
 6   afm_hrs_last_insp  35427 non-null  float64
 7   type_fly           35427 non-null  object 
 8   dprt_state         35427 non-null  object 
 9   rwy_len            35427 non-null  float64
 10  rwy_width          35427 non-null  float64
 11  ev_type            35427 non-null  object 
 12  ev_city            35427 non-null  object 
 13  ev_state           35427 non-null  object 
 14  ev_country         35427 non-null  object 
 15  ev_highest_injury  35427 non-null  object 
 16  sky_cond_ceil      35

# Data Preprocessing: 
One-hot encoding, Scaling, and Standardizing the data

Using the `final_df`, we created a variable `X` for all predictors and `y` for the response variable: `Injury`.

In [16]:
X = final_df.drop("Injury", axis = 1).copy()
y = final_df["Injury"].copy()

### One hot encoding and tranforming our `X` and `y`:
We wanted to first one hot encode `X` and tranform it into a sparse matrix. Using the Label Encoder from sklearn we `y` into a binary array. 

In [17]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop="first")
encoder.fit(X)
X = encoder.transform(X)
X

<35427x25112 sparse matrix of type '<class 'numpy.float64'>'
	with 646102 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
y

array([0, 0, 0, ..., 1, 1, 1])

### Training and Testing Split
For training and testing, we gathered 80% of data for the training and 20% for testing data using `scikit-learn`. So far, we have not used cross validation.

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Standardizing and scaling our data:
From sklearn.preprocessing, the StandardScaler allowed us standardize our data. We wanted to compare performance metrics between the raw data vs scaled and standardized data.

In [20]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(class_weight="balanced")
logisticRegr.fit(X_train, y_train)
y_hat = logisticRegr.predict(X_test)

In [21]:
from sklearn import metrics as mt
accuracy_raw = mt.accuracy_score(y_test,y_hat)
conf_raw = mt.confusion_matrix(y_test,y_hat)
print("The accuracy for the raw data is: ",accuracy_raw)
print(conf_raw)

The accuracy for the raw data is:  0.9992943832909963
[[3330    5]
 [   0 3751]]


In [22]:
from sklearn.preprocessing import StandardScaler

scl_obj = StandardScaler(with_mean=False)
scl_obj.fit(X_train) 

X_train_scaled = scl_obj.transform(X_train) 
X_test_scaled = scl_obj.transform(X_test)

In [23]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(class_weight="balanced")
logisticRegr.fit(X_train_scaled, y_train)
y_hat = logisticRegr.predict(X_test_scaled)

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [25]:
from sklearn import metrics as mt
accuracy_stan = mt.accuracy_score(y_test,y_hat)
conf_stan = mt.confusion_matrix(y_test,y_hat)
print("The accuracy for the standardized and scaled data is:", accuracy_stan)
print(conf_stan)

The accuracy for the standardized and scaled data is: 0.9983065198983911
[[3334    1]
 [  11 3740]]


As we can see, the accuracy is too high. `ev_highest_injury` is a categorical level that indicates the highest level of injury among all injuries sustained as a result of the event. We found that `ev_highest_injury` must be highly correlated our response variable, `Injury`. By removing `ev_highest_injury`, our accuracy would measure better moving forward.

Here, we looked to see if there were any variables that may be too coorlated with our regression. We got a 99% prediction rate which told us we had a variable that should be considered "unknown" We removed ev_highest_injury which should be treated as unknown. This fixes our issues of too high of accurcy.

# Final Models: USE THIS HOMIES

In [26]:
df = final_df.copy()
del df['ev_highest_injury']

In [27]:
X = df.drop("Injury", axis = 1).copy()
y = df["Injury"].copy()

In [ ]:
#one hot encoding with proper model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop="first")
encoder.fit(X)
X = encoder.transform(X)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
# scale attributes by the training set
scl_obj = StandardScaler(with_mean=False)
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test)

# Logistic Regression

In [ ]:
#scaled
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(class_weight="balanced",solver='liblinear',penalty = 'l2')
logisticRegr.fit(X_train_scaled, y_train)
y_hat = logisticRegr.predict(X_test_scaled)

from sklearn import metrics as mt
acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print(acc)
print(conf)

In [ ]:
#NOT Scaled
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(class_weight="balanced",solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train, y_train)
y_hat = logisticRegr.predict(X_test)

from sklearn import metrics as mt
acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print(acc)
print(conf)

In [ ]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'Injury' in final_df:
    y = final_df['Injury'].values # get the labels we want
    del final_df['Injury'] # get rid of the class label
    X = final_df.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

to do:
one hot encoding
avoid confounding variables - this causes an issue with feature importance
check for highly corrilated variables - 
use a confusion matrix
scale our data
large diff in KDE for support vectors - it falls along the decision bountry vs the read data


In [ ]:
# from sklearn.preprocessing import StandardScaler, OneHotEncoder

# # Define which columns should be encoded vs scaled
# #Categorical columns to convert to one hot encoding
# columns_to_encode = ['acft_make',"acft_model","acft_category", "damage",
#                      "far_part","type_fly","dprt_state",
#                      "ev_type","ev_city","ev_state","ev_country",
#                      "ev_highest_injury","sky_cond_ceil",
#                      "sky_cond_nonceil","wind_vel_ind","wx_int_precip",
#                      "phase_flt_spec"]
# #Continuous columns to be scaled
# columns_to_scale  = ['cert_max_gr_wt', 'afm_hrs_last_insp',
#                      'rwy_len',"rwy_width"]

# # Instantiate encoder/scaler
# scaler = StandardScaler()
# ohe = OneHotEncoder(drop="first")

# # Scale and Encode Separate Columns
# scaled_columns = scaler.fit(X[columns_to_scale]) 
# a = scaled_columns.transform(X)
# encoded_columns = ohe.fit(X[columns_to_encode])
# b = encoded_columns.transform(X)
# # Concatenate (Column-Bind) Processed Columns Back Together
# c = np.concatenate([a, b], axis=1)